In [3]:
import pandas as pd
import matplotlib.pyplot as plt


from scholarly import scholarly
import re
import networkx as nx

from coauthorship import *

%matplotlib inline

In [5]:

name = 'Marine Lasbleis'
short_name = "M Lasbleis"

search_query = scholarly.search_author('Marine Lasbleis')
author = next(search_query)#.fill()
print(author)

{&#39;affiliation&#39;: &#39;MSCA-IF Fellow, Université de Nantes&#39;,
 &#39;citedby&#39;: 125,
 &#39;email&#39;: &#39;@univ-nantes.fr&#39;,
 &#39;filled&#39;: False,
 &#39;id&#39;: &#39;QOi1XIEAAAAJ&#39;,
 &#39;interests&#39;: [&#39;geodynamics&#39;, &#39;planetary cores&#39;, &#39;2-phase flow&#39;],
 &#39;name&#39;: &#39;Marine Lasbleis&#39;,
 &#39;url_picture&#39;: &#39;https://scholar.google.com/citations?view_op=medium_photo&amp;user=QOi1XIEAAAAJ&#39;}


In [6]:
author = author.fill() # add info about co authors and publications

In [ ]:

author_list = {}



relationships = pd.DataFrame(0, index=[short_name], columns=[short_name])
for pub in author.publications:
    #print(pub)
    if float(pub.bib["cites"])> 0:  #only the papers that have been cited at least once
        pub_complete = pub.fill()
        print(pub_complete.bib["title"])
        authors = pub_complete.bib["author"]
        authors = re.split(" and ", authors)
        #print(authors)
        single_paper = []
        for author_name in authors: 
            single_author = format_name_authors(author_name)
            author_list = add_author(author_list, single_author)
            single_paper.append(single_author)
            if single_author not in relationships.columns:
                relationships[single_author] = 0.
                #print(relationships)
                new_line = pd.DataFrame(0, index=[single_author], columns=relationships.columns)
                relationships = relationships.append(new_line)
        for author1 in single_paper:
            for author2 in single_paper:
                if author1 != author2:
                    relationships[author1][author2] += 1
        # add authors in columns and lines in the pandaframe relationships
        # add the relationships for each couples in the author list (if author list is longer than 1!)


print(author_list)    
print(relationships)